# Prevendo Preços de Casas

## Carregando Pacotes

In [1]:
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPool2D, Activation, Dropout, Flatten, Input
from keras.optimizers import Adam
from keras import backend as K
from keras.layers import concatenate
import pandas as pd
import numpy as np
import glob
import cv2
import locale
import os

Using TensorFlow backend.


In [2]:
K.image_data_format()

'channels_last'

## Carregando Dataset (Atributos e imagens)

In [29]:
# Criação dos labels das colunas e carregando dataset
cols = ['bedrooms', 'bathrooms', 'area', 'zipcode', 'price']
df = pd.read_csv("/home/matheusjerico/Documentos/DSA/05. MachineLearning/Cap-12-IntroduçãoDeepLearning/Houses_dataset/HousesInfo.txt", sep = " ", header=None, names=cols)

# Determinando os zip code unicos e quantos tem de cada
zipcodes = df['zipcode'].value_counts().keys().tolist()
counts = df['zipcode'].value_counts().tolist()

# Loop para cada zip code e sua respectiva quantidade
for (zipcode, count) in zip(zipcodes, counts):
    # remover zipcodes que possuem menos de 25 counts
    if count < 25:
        idxs = df[df['zipcode'] == zipcode].index
        df.drop(idxs, inplace=True)

# Carregando imagens
inputPath = "/home/matheusjerico/Documentos/DSA/05. MachineLearning/Cap-12-IntroduçãoDeepLearning/Houses_dataset"
images= []

for i in df.index.values:
    # Achar as 4 imagens de cada casa, e garantir estar sempre na mesma ordem
    basePath = os.path.sep.join([inputPath,"{}_*".format(i+1)])
    housePaths=sorted(list(glob.glob(basePath)))
    
    # inicializa a list e preenche com a nova imagem que contem 4 imagens
    inputImages = []
    outputImage = np.zeros((64, 64, 3), dtype='uint8')

    for housePath in housePaths:
        # carregue a imagem de entrada, redimensione-a para 32 32 e, em seguida,
        # atualiza a lista de imagens de entrada
        image = cv2.imread(housePath)
        image = cv2.resize(image, (32,32))
        inputImages.append(image)
    
    # telha as quatro imagens de entrada na imagem de saída como o primeiro
    # imagem vai no canto superior direito, a segunda imagem no
    # canto superior esquerdo, a terceira imagem no canto inferior direito,
    # e a imagem final no canto inferior esquerdo
    outputImage[0:32, 0:32] = inputImages[0]
    outputImage[0:32, 32:64] = inputImages[1]
    outputImage[32:64, 32:64] = inputImages[2]
    outputImage[32:64, 0:32] = inputImages[3]
                           
    # adicione a imagem lado a lado ao nosso conjunto de imagens a rede será treinado em
    images.append(outputImage)

images = np.array(images)
images = images / 255.0

## Construindo o Dataset de treino e teste

In [30]:
# Construindo o dataset de treino e teste
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

In [31]:
trainAttrX.head(3)

,bedrooms,bathrooms,area,zipcode,price
148,5,3.5,4245,92677,1895000.0
337,2,2.0,1176,92276,135000.0
444,3,3.0,2800,93510,630000.0


## Regularizando a variável target

In [32]:
maxprice = trainAttrX['price'].max()
y_train = trainAttrX['price'] / maxprice
y_test = testAttrX['price'] / maxprice

## Manipulando os atributos das casas

- Pré Processamento

In [33]:
# Colunas de valores continuos do dataset
continuos = ['bedrooms','bathrooms','area']

# Escalonando os dados em MinMax no range [0,1]
cs = MinMaxScaler()
trainContinuos = cs.fit_transform(trainAttrX[continuos])
testContinuos = cs.transform(testAttrX[continuos])

# One-hot code para variáveis categóricas
zipBinarizer = LabelBinarizer().fit(df['zipcode'])
trainCategorical = zipBinarizer.transform(trainAttrX['zipcode'])
testCategorical = zipBinarizer.transform(testAttrX['zipcode'])

# Reconstruindo dataset com de treino e teste com as variáveis categoricas e continuas
trainAttrX = np.hstack([trainCategorical, trainContinuos])
testAttrX = np.hstack([testCategorical, testContinuos])


## Criando o modelo


In [34]:
# Criando o modelo para MLP e CNN

# Criando MLP
model_mlp = Sequential()
model_mlp.add(Dense(8, input_dim=trainAttrX.shape[1], activation='relu'))
model_mlp.add(Dense(4, activation='relu'))
    
# Criando CNN
# Assumindo que o beckend é TensorFlow
filters=(16, 32, 64)
inputShape = (64, 64, 3)
chanDim = -1

# Definindo model input
inputs = Input(shape=inputShape)

# Loop para cada filtro
for (i, f) in enumerate(filters):
    
    # primeira camada tem que setar o 'input'
    if i==0:
        x = inputs
    
    # CONV => RELU => BATCH => MAXPOOLING
    x = Conv2D(f,(3,3), padding='same')(x)
    x = Activation('relu')(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPool2D(pool_size=(2,2))(x)

# achar o volume, então FC => RELU => BN => DROPOUT
x = Flatten()(x)
x = Dense(16)(x)
x = Activation("relu")(x)
x = BatchNormalization(axis=chanDim)(x)
x = Dropout(0.5)(x)

# aplica outra camada FC, esta para corresponder ao número de nós saindo do MLP
x = Dense(4)(x)
x = Activation('relu')(x)

model_cnn = Model(inputs, x)


# Combinando os modelos
combinedInput = concatenate([model_mlp.output, model_cnn.output])

# Ultima layers deve ter 1 node linear
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)


# nosso modelo final aceita dados categóricos / numéricos no MLP
# input e images na entrada CNN, produzindo um único valor (o preço previsto da casa)
model = Model(inputs=[model_mlp.input, model_cnn.input], outputs=x)

## Compilando e Fit no modelo


In [35]:
# Parametro de otimização
opt = Adam(lr=0.001, decay=0.001/200)

# Compilando
model.compile(loss='mean_absolute_percentage_error', optimizer=opt)

# Fit
model.fit([trainAttrX, trainImagesX], y_train, 
          validation_data=([testAttrX, testImagesX], y_test),
          epochs=200, batch_size=8)

Train on 271 samples, validate on 91 samples
Epoch 1/200
271/271 [==============================] - 7s 24ms/step - loss: 1151.8355 - val_loss: 1300.5865
Epoch 2/200
271/271 [==============================] - 3s 11ms/step - loss: 645.0102 - val_loss: 932.5326
Epoch 3/200
271/271 [==============================] - 3s 11ms/step - loss: 357.5838 - val_loss: 579.3071
Epoch 4/200
271/271 [==============================] - 2s 9ms/step - loss: 361.9901 - val_loss: 238.6071
Epoch 5/200
271/271 [==============================] - 2s 9ms/step - loss: 245.4629 - val_loss: 228.6747
Epoch 6/200
271/271 [==============================] - 2s 9ms/step - loss: 210.7334 - val_loss: 163.0280
Epoch 7/200
271/271 [==============================] - 2s 9ms/step - loss: 178.8919 - val_loss: 122.2087
Epoch 8/200
271/271 [==============================] - 2s 9ms/step - loss: 174.3149 - val_loss: 115.6707
Epoch 9/200
271/271 [==============================] - 2s 9ms/step - loss: 127.1061 - val_loss: 87.7671
Epoch 

Epoch 79/200
271/271 [==============================] - 2s 9ms/step - loss: 26.0798 - val_loss: 24.9026
Epoch 80/200
271/271 [==============================] - 2s 9ms/step - loss: 25.4383 - val_loss: 26.3145
Epoch 81/200
271/271 [==============================] - 2s 9ms/step - loss: 28.1624 - val_loss: 27.5193
Epoch 82/200
271/271 [==============================] - 2s 9ms/step - loss: 30.0474 - val_loss: 28.6431
Epoch 83/200
271/271 [==============================] - 2s 9ms/step - loss: 28.1447 - val_loss: 27.0263
Epoch 84/200
271/271 [==============================] - 2s 9ms/step - loss: 25.9613 - val_loss: 26.3926
Epoch 85/200
271/271 [==============================] - 2s 9ms/step - loss: 24.8710 - val_loss: 25.3008
Epoch 86/200
271/271 [==============================] - 2s 8ms/step - loss: 26.5678 - val_loss: 25.7656
Epoch 87/200
271/271 [==============================] - 2s 9ms/step - loss: 26.2225 - val_loss: 25.5251
Epoch 88/200
271/271 [==============================] - 2s 8ms/s

Epoch 157/200
271/271 [==============================] - 3s 12ms/step - loss: 21.2053 - val_loss: 23.5876
Epoch 158/200
271/271 [==============================] - 2s 9ms/step - loss: 22.0368 - val_loss: 24.5103
Epoch 159/200
271/271 [==============================] - 3s 10ms/step - loss: 20.4805 - val_loss: 26.4694
Epoch 160/200
271/271 [==============================] - 3s 11ms/step - loss: 20.2434 - val_loss: 26.2332
Epoch 161/200
271/271 [==============================] - 3s 12ms/step - loss: 21.5987 - val_loss: 25.2065
Epoch 162/200
271/271 [==============================] - 3s 12ms/step - loss: 22.2500 - val_loss: 25.8188
Epoch 163/200
271/271 [==============================] - 3s 13ms/step - loss: 20.6875 - val_loss: 24.9807
Epoch 164/200
271/271 [==============================] - 3s 11ms/step - loss: 20.4057 - val_loss: 26.3707
Epoch 165/200
271/271 [==============================] - 3s 10ms/step - loss: 20.8231 - val_loss: 25.8436
Epoch 166/200
271/271 [========================

## Avaliando o modelo


In [36]:
# Prevendo
preds = model.predict([testAttrX, testImagesX])

# Diferença entre predição e valor real
diff = preds.flatten() - y_test
percentDiff = (diff / y_test) * 100
absPercentDiff = np.abs(percentDiff)

# Media e desvio padrão
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# Mostrar estatísticas do modelo
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
    locale.currency(df["price"].mean(), grouping=True),
    locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 79.99%, std: 244.43%
